In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch09. 8장 chroma→pinecone을 활용한 RAG 구현 </font></b>

# 벡터DB : Chroma vs Pinecone

- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    - https://www.pinecone.io 에서 api key 생성 → .env에 추가(PINECONE_API_KEY 등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=200)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [5]:
# embedding : OpenAI API text-embedding-3-large

from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [7]:
%%time

# pinecone vector database 저장
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

# 데이터를 처음 업로드할 때
index_name = 'tax-index'
# database = PineconeVectorStore.from_documents(documents=document_list,
#                                               embedding=embedding,
#                                               index_name=index_name)

# 업로드한 벡터DB를 가져올 때
database = PineconeVectorStore(embedding=embedding, # 질문을 임베딩하여 유사도 검색
                               index_name=index_name)

CPU times: total: 0 ns
Wall time: 1e+03 μs


# 2. 답변 생성을 위한 Retrieval

In [8]:
query = '연봉이 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3) # 기본 k 값 : 4

In [12]:
# retrieved_docs[0].page_content

retrieved_doc = '\n\n---\n\n'.join([doc.page_content for doc in retrieved_docs])

In [13]:
# 두칸 위에있는거랑 완벽하게 같음

retriever = database.as_retriever(search_kwargs={'k':3})
retrieved_docs = retriever.invoke(query)

# 3. 답변 생성

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano')

In [14]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면

from langchain_upstage import ChatUpstage

llm = ChatUpstage(model = 'solar-pro2',
                  reasoning_effort='high') # 느리지만 더 깊게 추론함 (low, medium)

In [15]:
prompt = f'''[identity]
-당신은 최고의 한국 소득세법 전문가입니다
-[context]를 참고해서 사용자의 질문에 답변해주세요
-[context]는 다음과 같습니다
{retrieved_doc}
-질문 : {query}'''

In [16]:
ai_message = llm.invoke(prompt)

In [17]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세 계산은 다음과 같이 진행됩니다. 단, **2024년 기준** 및 제공된 [context]를 기반으로 핵심 규정을 적용하였습니다.

---

### 1. **근로소득금액 계산 (제20조, 제47조)**
- **총급여액**: 50,000,000원  
- **근로소득공제** (제47조):  
  - 총급여액 5,000만원에 대한 공제액은 **2,000만원**으로 제한됨 (제47조 1항 단서).  
  - **근로소득금액** = 총급여액 - 근로소득공제 = 50,000,000 - 20,000,000 = **30,000,000원**

---

### 2. **종합소득과세표준 계산**
- **기본공제** (제52조 등, [context] 미제공 시 일반 규정 적용):  
  - 본인 기본공제: 1,500,000원  
  - (추가 공제 항목 없음 가정)  
- **종합소득과세표준** = 근로소득금액 - 기본공제 = 30,000,000 - 1,500,000 = **28,500,000원**

---

### 3. **산출세액 계산 (누진세율 적용)**
2024년 종합소득세 누진세율 기준:  
| 과세표준 구간 | 세율 | 누진공제 |
|----------------|------|----------|
| 1,200만원 이하 | 6%   | -        |
| 1,200만원 초과 ~ 4,600만원 이하 | 15%  | 108만원  |
| 4,600만원 초과 ~ 8,800만원 이하 | 24%  | 522만원  |

- **산출세액** = (28,500,000 × 15%) - 108만원 = **3,245,000원**

---

### 4. **세액공제 및 감면**
- **근로소득세액공제** (제55조):  
  - 산출세액의 55% 공제 (최대 72만원).  
  - 공제액 = 3,245,000 × 55% = 1,784,750원 → **72만원** 적용.  
- **최종 산출세액** = 3,245,000 - 720,000 = **2,525,000